# Stitching images using translation only
Leanne Friedrich 5/22/21

## initialization

In [2]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED = False

In [3]:
import os
import cv2
import sys
sys.path.append("../")
sys.path.append("../../")
from py.tools.config import cfg
import py.tools.logs as logs
import py.pic_stitch_bas as sb
from matplotlib import pyplot as plt
LOGGERDEFINED = logs.openLog('pic_stitch_bas.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=False)

## test stitching

In [98]:
run ../../tests/test_stitch_bas

INFO: 
xs
-----
INFO: 
horiz
-----
INFO: 
vert
-----


.

INFO: 
xs
-----
INFO: 
horiz
-----
INFO: 
vert
-----


.

KeyboardInterrupt: 

# stitch images

In [116]:
ss = sb.stitchSorterTriple(os.path.join(cfg.path.server, r'tripleLines\LapRD_LapRD_visc\I_4.00_S_2.25\I_4.00_S_2.25_220428'))
ss.printGroups()

INFO: 
HIB
-----
INFO: 
HIPh
-----
INFO: 
HIPxs
-----
INFO: HIPxs_0.500_0: Stitch: tripleLinesXS_+y_0.500_HIPxs_0_stitch_1_I_4.00_S_2.25_220428_174910_5.png
	Stills:tripleLinesXSPics_Basler camera_I_4.00_S_2.25_220428_174910_5.png

INFO: HIPxs_0.500_1: Stitch: tripleLinesXS_+y_0.500_HIPxs_1_stitch_1_I_4.00_S_2.25_220428_174913_3.png
	Stills:tripleLinesXSPics_Basler camera_I_4.00_S_2.25_220428_174913_3.png

INFO: HIPxs_0.500_2: Stitch: tripleLinesXS_+y_0.500_HIPxs_2_stitch_1_I_4.00_S_2.25_220428_174916_0.png
	Stills:tripleLinesXSPics_Basler camera_I_4.00_S_2.25_220428_174916_0.png

INFO: HIPxs_0.500_3: Stitch: tripleLinesXS_+y_0.500_HIPxs_3_stitch_1_I_4.00_S_2.25_220428_174918_8.png
	Stills:tripleLinesXSPics_Basler camera_I_4.00_S_2.25_220428_174918_8.png

INFO: HIPxs_0.625_0: Stitch: tripleLinesXS_+y_0.625_HIPxs_0_stitch_1_I_4.00_S_2.25_220428_174923_7.png
	Stills:tripleLinesXSPics_Basler camera_I_4.00_S_2.25_220428_174923_7.png

INFO: HIPxs_0.625_1: Stitch: tripleLinesXS_+y_0.625_HIPx

In [7]:
sb.stitchRecursive(os.path.join(cfg.path.server, r'tripleLines\mineral_LapRD_visc\I_M7_S_3.00\I_M7_S_3.00_220426'), st='HIPxs', overwrite=True)

INFO: 
--------------
--------------
//cfs2e.nist.gov/642/internal/3D Printing Lab/Shopbot/videos/tripleLines\mineral_LapRD_visc\I_M7_S_3.00\I_M7_S_3.00_220426
-------
INFO: 
HIPxs
-----


In [64]:
import fileHandling as fh

def sortStitch(printFolder:str, real:str):
    good = []
    bad = []
    pfd = fh.printFileDict(printFolder)
    for s in pfd.stitch:
        if real in s:
            good.append(s)
        else:
            bad.append(s)
    return good,bad

def sorts(fHOB:str, fHOC:str, real:str):
    goodHOB,badHOB = sortStitch(fHOB, real)
    if len(goodHOB)==0:
        goodHOC1,badHOC1 = sortStitch(fHOC, real)
        if len(goodHOC1)>0:
            bn1 = os.path.basename(fHOB)
            bn2 = os.path.basename(fHOC)
            oldname = goodHOC1[0]
            newname = goodHOC1[0].replace(bn2, bn1)
            os.rename(oldname, newname)
            print(f'Renamed {fh.twoBN(oldname)} to {fh.twoBN(newname)}')
    return badHOB


def switchCB(folder:str):
    if not fh.isSubFolder(folder):
        return
    fl = sb.fileList(folder)
    for i,fHOB in fl.HOBfolders.items():
        fHOC = fl.HOCfolders[i]
        badHOB = sorts(fHOB, fHOC, 'HOB')
        badHOC = sorts(fHOC, fHOB, 'HOC')
        goodHOB,badHOB = sortStitch(fHOB, 'HOB')
        if len(badHOB)>0:
            for file in badHOB:
                os.remove(file)
        goodHOC,badHOC = sortStitch(fHOC, 'HOC')
        if len(badHOC)>0:
            for file in badHOC:
                os.remove(file)
    for i,fVB in fl.VBfolders.items():
        fVC = fl.VCfolders[i]
        badVB = sorts(fVB, fVC, 'VB')
        badVC = sorts(fVC, fVB, 'VC')

topFolder = r'\\cfs2e.nist.gov\642\internal\3D Printing Lab\Shopbot\videos\tripleLines'
for typeFolder0 in os.listdir(topFolder):
    typeFolder = os.path.join(topFolder, typeFolder0)
    for sampleFolder0 in os.listdir(typeFolder):
        sampleFolder = os.path.join(typeFolder, sampleFolder0)
        for subFolder0 in os.listdir(sampleFolder):
            subFolder = os.path.join(sampleFolder, subFolder0)
            try:
                switchCB(subFolder)
            except:
                print(subFolder)
                pass

\\cfs2e.nist.gov\642\internal\3D Printing Lab\Shopbot\videos\tripleLines\PDMSS_LapRD_visc\I_PDMSS12.5_S_4.00\I_PDMSS12.5_S_4.00_220503
